In [1]:
from importlib.util import module_from_spec, spec_from_file_location
from pathlib import Path

import pandas as pd

In [2]:
def import_database_module():
    # Determine the path to the 'actions' directory relative to the notebook's location
    actions_path = Path.cwd().parent / "actions"
    database_file = actions_path / "database.py"

    # Create a module specification
    spec = spec_from_file_location("database", database_file)

    # Create a module based on the specification
    database = module_from_spec(spec)

    # Load the module
    spec.loader.exec_module(database)

    return database


database = import_database_module()

---

In [3]:
def get_all_user_account_card_info(db) -> pd.DataFrame:
    query = """
        SELECT
            u.name AS username,
            u.password,
            c.number AS card_number,
            a.number AS account_number,
            a.sort_code
        FROM users u
        JOIN accounts a ON u.id = a.user_id
        JOIN cards c ON u.id = c.user_id AND a.id = c.account_id
        WHERE a.type = 'current'
    """
    results = db.run_query(query, one_record=False)
    columns = ["username", "password", "card_number", "account_number", "sort_code"]
    df = pd.DataFrame(results, columns=columns)
    return df


Database = database.Database
db = Database()

try:
    user_info_df = get_all_user_account_card_info(db)
    display(user_info_df)
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    db.__exit__(None, None, None)

2024-10-02 18:33:16,755     INFO database  - Creating new in-memory database
2024-10-02 18:33:16,757    DEBUG database  - Creating table 'users'
2024-10-02 18:33:16,759    DEBUG database  - Creating table 'accounts'
2024-10-02 18:33:16,762    DEBUG database  - Creating table 'transactions'
2024-10-02 18:33:16,763    DEBUG database  - Creating table 'payees'
2024-10-02 18:33:16,764    DEBUG database  - Creating table 'cards'
2024-10-02 18:33:16,770    DEBUG database  - Creating table 'branches'
2024-10-02 18:33:16,771    DEBUG database  - Creating table 'advisors'
2024-10-02 18:33:16,772    DEBUG database  - Creating table 'appointments'
2024-10-02 18:33:16,772     INFO database  - Database schema created
2024-10-02 18:33:16,776     INFO database  - Attempting to load data for table 'appointments' from /Users/leo/Git/moad2/data/source/appointments.json
2024-10-02 18:33:16,777     INFO database  - Inserted 210 records into table 'appointments'
2024-10-02 18:33:16,778     INFO database  -

,username,password,card_number,account_number,sort_code
0,Hari Seldon,seldon123,1234567812345678,23456789,123456
1,Salvor Hardin,hardin123,9999000011112222,34567890,234567
2,Gaal Dornick,dornick123,4567890145678901,78901234,345678


2024-10-02 18:33:16,796     INFO database  - Database connection closed
